In [17]:
import torch
from torchvision import models
from torch.utils.mobile_optimizer import optimize_for_mobile

In [69]:
model = models.shufflenet_v2_x0_5()
model.fc.weight.device

device(type='cpu')

In [70]:
x = torch.ones(1, 3, 224, 224)

In [71]:
model(x).shape

torch.Size([1, 1000])

# torch.jit.trace

## gpu

In [72]:
trace_model = torch.jit.trace(model.cuda(), example_inputs=x.cuda())
trace_model.fc.weight.device

device(type='cuda', index=0)

In [74]:
torch.jit.save(trace_model, r"shufflenet_v2_x0_5.gpu.torchscript")
# trace_model.save("shufflenet_v2_x0_5.torchscript")

In [75]:
trace_model_ = torch.jit.load(r"shufflenet_v2_x0_5.gpu.torchscript")
trace_model_.fc.weight.device

device(type='cuda', index=0)

In [77]:
trace_model_(x.cuda()).shape

torch.Size([1, 1000])

## cpu

In [78]:
trace_model = torch.jit.trace(model.cpu(), example_inputs=x)

In [79]:
torch.jit.save(trace_model, r"shufflenet_v2_x0_5.cpu.torchscript")
# trace_model.save("shufflenet_v2_x0_5.torchscript")

In [80]:
trace_model_ = torch.jit.load(r"shufflenet_v2_x0_5.cpu.torchscript")
trace_model_.fc.weight.device

device(type='cpu')

In [81]:
trace_model_(x).shape

torch.Size([1, 1000])

# optimize_for_mobile 只能使用cpu的模型

## backend="CPU"

In [82]:
optimize_for_mobile(trace_model, backend="CPU") \
    ._save_for_lite_interpreter(r"shufflenet_v2_x0_5-optimize-CPU.torchscript")

In [83]:
trace_model_mobile = torch.jit.load(r"shufflenet_v2_x0_5-optimize-CPU.torchscript")

In [84]:
trace_model_mobile(x.cpu()).shape

torch.Size([1, 1000])

## backend="Vulkan" 需要编译支持Vulcan的Pytorch

In [85]:
optimize_for_mobile(trace_model, backend="Vulkan") \
    ._save_for_lite_interpreter("shufflenet_v2_x0_5-optimize-Vulkan.torchscript")

RuntimeError: 0 INTERNAL ASSERT FAILED at "..\\torch\\csrc\\jit\\ir\\alias_analysis.cpp":621, please report a bug to PyTorch. We don't have an op for vulkan_prepack::create_linear_context but it isn't a special case.  Argument types: Tensor, Tensor, 

Candidates:

## backend="Metal" 需要编译支持Metal的Pytorch

In [86]:
optimize_for_mobile(trace_model, backend="Metal") \
    ._save_for_lite_interpreter("shufflenet_v2_x0_5-optimize-Metal.torchscript")

RuntimeError: 0 INTERNAL ASSERT FAILED at "..\\torch\\csrc\\jit\\ir\\alias_analysis.cpp":621, please report a bug to PyTorch. We don't have an op for metal_prepack::linear_prepack but it isn't a special case.  Argument types: Tensor, Tensor, NoneType, NoneType, 

Candidates: